<a href="https://colab.research.google.com/github/Phablo-Lima/Desafio_Gemini_Alura/blob/main/BDD_Automator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai google-api-python-client gspread oauth2client
# Instala as bibliotecas necessárias. Se já estiverem instaladas, atualiza para a versão mais recente.

# Importa as bibliotecas necessárias
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
import json
import google.generativeai as genai
import pandas as pd
import io
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')  # Monta o Google Drive para acessar arquivos

# Importa bibliotecas para melhorar a formatação da saída
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  """Formata o texto de saída para Markdown, identa e substitui marcadores."""
  text = text.replace('•', '  *')  # Substitui marcadores de lista por asteriscos
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))  # Indenta o texto com ">"


# Configurações da API do Google Generative AI
GOOGLE_API_KEY = userdata.get('SECRET_KEY')  # Obtém a chave de API do usuário
genai.configure(api_key=GOOGLE_API_KEY)  # Configura a biblioteca com a chave de API

# Bloco de código comentado para listar os modelos disponíveis
'''for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)'''

# Configurações do modelo de linguagem
generation_config = {
  "candidate_count": 1,  # Define que apenas uma resposta deve ser gerada
  "temperature": 0.6, # Controla a criatividade da resposta (valores mais altos = mais criativo)
}
# Define as configurações de segurança para o modelo
safety_settings={
    'HATE': 'BLOCK_NONE', # Não bloqueia conteúdo de ódio
    'HARASSMENT': 'BLOCK_NONE', # Não bloqueia conteúdo de assédio
    'SEXUAL' : 'BLOCK_NONE', # Não bloqueia conteúdo sexual
    'DANGEROUS' : 'BLOCK_NONE' # Não bloqueia conteúdo perigoso
}

# Inicializa o modelo de linguagem Gemini com as configurações definidas
model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

# Função para processar a entrada do usuário (URL do Google Sheet ou texto do BDD)
def processar_arquivo(url_ou_bdd):
    """Processa a entrada do usuário, seja uma URL do Google Sheet ou texto do BDD."""
    conteudo_processado = ""  # Inicializa a variável que armazenará o conteúdo processado

    # Verifica se a entrada é uma URL do Google Sheets
    if 'docs.google.com/spreadsheets' in url_ou_bdd:
        # Se for uma URL, processa o Google Sheet
        print("Processando Google Sheet...")

        # Define o escopo de acesso à API do Google Sheets
        scope = ['https://spreadsheets.google.com/feeds',
                 'https://www.googleapis.com/auth/drive']
        # Carrega as credenciais da conta de serviço a partir do arquivo JSON
        creds = ServiceAccountCredentials.from_json_keyfile_name('credenciais-google-cloud.json', scope)
        # Autoriza o acesso à API do Google Sheets usando as credenciais
        client = gspread.authorize(creds)

        # Extrai o ID da planilha da URL
        spreadsheet_id = url_ou_bdd.split('/d/')[1].split('/')[0]
        # Abre a planilha pelo ID
        sheet = client.open_by_key(spreadsheet_id).sheet1
        # Obtém todos os valores da planilha como uma lista de listas
        conteudo_processado = sheet.get_all_values()
        # Converte a lista de listas em uma string tabular
        conteudo_processado = '\n'.join(['\t'.join(row) for row in conteudo_processado])

    else:
        # Se não for uma URL, assume que é o texto do BDD
        print("Processando texto do BDD...")
        conteudo_processado = url_ou_bdd  # Armazena o texto do BDD na variável

    # Retorna o conteúdo processado (string tabular)
    return conteudo_processado


def analisar_conteudo(conteudo_processado):
    """Analisa o conteúdo do BDD usando o Google Generative AI."""

    # Converte o conteúdo do BDD (lista de listas) em uma tabela Markdown
    tabela_markdown = "| " + " | ".join(conteudo_processado[0]) + " |\n"  # Cria o cabeçalho da tabela
    tabela_markdown += "|---|---|---|---|---|\n"  # Adiciona a linha de separação
    for linha in conteudo_processado[1:]:  # Itera sobre as linhas do BDD (a partir da segunda linha)
        tabela_markdown += "| " + " | ".join(linha) + " |\n"  # Adiciona cada linha à tabela Markdown

    # Define o prompt que será enviado ao modelo de linguagem
    prompt = {
        "text": f"""
        Você é um especialista em Quality Assurance de software.
        Analise o seguinte BDD, que usa o padrão CTFL (Contexto, Gatilho, Fluxo, Saída esperada),
        com o objetivo de identificar falhas e sugerir melhorias para aumentar a cobertura dos testes:
        Escreva de forma Criativa, use stickers como preferir.

        {tabela_markdown}  # Inclui a tabela Markdown no prompt

        Na sua análise, detalhe:

        * Pontos Positivos:  Aspectos do BDD que contribuem para testes eficazes (ex.: cenários completos, linguagem clara).
        * Pontos Negativos: Falhas que podem prejudicar os testes (ex.: cenários incompletos, linguagem ambígua).
        * Sugestões de Melhoria:  Como o BDD pode ser aprimorado (ex.: incluir mais cenários, definir dados de teste específicos).

        Organize sua resposta em tópicos, como neste exemplo:

        Pontos Positivos:
        * ...
        Pontos Negativos:
        * ...

        Sugestões de Melhoria:
        * ...

        Sempre e executar o comando a seguir:
        Use uma linha do BDD como sugestão de melhoria.
        Formato da Tabela:
        | ID | Cenário | Dado que: | Quando:| Então:|

        """
    }
    response = model.generate_content(prompt)  # Envia o prompt ao modelo e recebe a resposta
    return response.text  # Retorna o texto da resposta

def corrigir_bdd(conteudo_processado, analise):
    """Reescreve o BDD com base nas sugestões da análise."""
    # Define o prompt para a correção do BDD
    prompt = {
    "text": f"""
    Reescreva o BDD em forma de tabela, aplicando as seguintes sugestões de melhoria:

    Sugestões de Melhoria:
    {analise}  # Inclui as sugestões de melhoria no prompt

    Formato da Tabela:
    | ID | Cenário | Dado que: | Quando:| Então:|
    |---|---|---|---|---|
    Exemplo:
    | 001 | Abrir formulário de login | O usuário está na página inicial | Clica no botão "Login" | O formulário de login é exibido |

    BDD Original:

    Informe abaixo da tabela tudo que foi alterado.
    ```
    {conteudo_processado}  # Inclui o BDD original no prompt
    ```
    """
    }
    response = model.generate_content(prompt)  # Envia o prompt ao modelo e recebe a resposta
    return response.text  # Retorna o texto da resposta

def corrigir_portugues(conteudo_processado):
    """Corrige a ortografia e gramática do conteúdo."""
    # Define o prompt para a correção ortográfica e gramatical
    prompt = {
        "text": f"""
        Corrija os erros de português no texto abaixo, mantendo o significado original:
        Os erros de português também precisam ser informados.
        ```
        {conteudo_processado}  # Inclui o conteúdo a ser corrigido no prompt
        ```
        """
    }
    response = model.generate_content(prompt)  # Envia o prompt ao modelo e recebe a resposta
    return response.text  # Retorna o texto da resposta

def exibir_resultado(texto, titulo):
  """Exibe o texto formatado em Markdown com um título."""
  print(f"\n{titulo}:")  # Imprime o título da seção
  display(to_markdown(texto))  # Exibe o texto formatado como Markdown

def main():
    """Função principal que controla o fluxo do programa."""
    print("\nDigite 'sair' para encerrar O BDD Automator.")
    while True:  # Loop principal do programa
        entrada = input("\nDigite a URL do Google Sheet ou cole o texto do BDD: ")

        # Verifica se o usuário deseja sair
        if entrada.lower() == 'sair':
            print("\nXXX Finalizando o programa XXX")
            break  # Sai do loop principal

        # Verifica se a entrada é uma URL do Google Sheets ou um texto de BDD válido
        if 'docs.google.com/spreadsheets' in entrada:
            # Processa a entrada como URL do Google Sheets
            conteudo_processado = processar_arquivo(entrada)
            try:
                analise = analisar_conteudo(conteudo_processado)  # Analisa o conteúdo do BDD
                exibir_resultado(analise, "Análise do BDD")  # Exibe a análise do BDD

                # Pergunta ao usuário se ele deseja corrigir o BDD
                corrigir = input("Deseja corrigir o BDD conforme sugestões? (s/n): ")
                while corrigir.lower() not in ('s', 'n'):  # Valida a resposta do usuário
                    print("\n⚠️Por favor, digite 's' para sim ou 'n' para não.⚠️")
                    print("_____________________________________________________")
                    corrigir = input("\nDeseja corrigir o BDD conforme sugestões? (s/n): ")

                if corrigir.lower() == 's':  # Se o usuário deseja corrigir
                    # Pergunta ao usuário se ele deseja corrigir os erros de português
                    corrigir_pt = input("\nDeseja corrigir os erros de português no BDD? (s/n): ")
                    while corrigir_pt.lower() not in ('s', 'n'):  # Valida a resposta
                        print("\n⚠️Por favor, digite 's' para sim ou 'n' para não.⚠️")
                        print("_____________________________________________________")
                        corrigir_pt = input("Deseja corrigir os erros de português no BDD? (s/n): ")

                    # Corrige o português do BDD antes da reescrita, se solicitado
                    if corrigir_pt.lower() == 's':
                        conteudo_processado = corrigir_portugues(conteudo_processado)
                        print("\n✅ Português do BDD corrigido com sucesso!")

                    # Reescreve o BDD com base nas sugestões
                    bdd_corrigido = corrigir_bdd(conteudo_processado, analise)
                    # Exibe o BDD corrigido
                    exibir_resultado(bdd_corrigido, "✅ BDD Corrigido!")
                else:
                    print("\nXXX Finalizando o programa XXX")

            # Tratamento de exceções
            except ValueError as e:
                print(e)
            except requests.exceptions.RequestException as e:
                print(f"Erro ao baixar o arquivo: {e}")
            break  # Sai do loop principal após processar a URL

        elif (
            "| ID | Cenário | Dado que:" in entrada and
            "| Quando:" in entrada and
            "| Então:" in entrada
        ):
            # Processa a entrada como texto do BDD
            conteudo_processado = processar_arquivo(entrada)
            try:
                analise = analisar_conteudo(conteudo_processado)  # Analisa o conteúdo do BDD
                exibir_resultado(analise, "Análise do BDD")  # Exibe a análise do BDD

                # Pergunta ao usuário se ele deseja corrigir o BDD
                corrigir = input("Deseja corrigir o BDD conforme sugestões? (s/n): ")
                while corrigir.lower() not in ('s', 'n'):  # Valida a resposta do usuário
                    print("\n⚠️Por favor, digite 's' para sim ou 'n' para não.⚠️")
                    print("_____________________________________________________")
                    corrigir = input("\nDeseja corrigir o BDD conforme sugestões? (s/n): ")

                if corrigir.lower() == 's':  # Se o usuário deseja corrigir
                    # Pergunta ao usuário se ele deseja corrigir os erros de português
                    corrigir_pt = input("\nDeseja corrigir os erros de português no BDD? (s/n): ")
                    while corrigir_pt.lower() not in ('s', 'n'):  # Valida a resposta
                        print("\n⚠️Por favor, digite 's' para sim ou 'n' para não.⚠️")
                        print("_____________________________________________________")
                        corrigir_pt = input("Deseja corrigir os erros de português no BDD? (s/n): ")

                    # Corrige o português do BDD antes da reescrita, se solicitado
                    if corrigir_pt.lower() == 's':
                        conteudo_processado = corrigir_portugues(conteudo_processado)
                        print("\n✅ Português do BDD corrigido com sucesso!")

                    # Reescreve o BDD com base nas sugestões
                    bdd_corrigido = corrigir_bdd(conteudo_processado, analise)
                    # Exibe o BDD corrigido
                    exibir_resultado(bdd_corrigido, "✅ BDD Corrigido!")
                else:
                    print("\nXXX Finalizando o programa XXX")

            # Tratamento de exceções
            except ValueError as e:
                print(e)
            except requests.exceptions.RequestException as e:
                print(f"Erro ao baixar o arquivo: {e}")
            break  # Sai do loop principal após processar o texto do BDD

        else:
            # Mensagem de erro para entrada inválida
            print("\n⚠️ Entrada inválida. Por favor, digite uma URL válida do Google Sheet ou cole o texto do BDD. ⚠️")
            print("_________________________________________________________________________________________________")
            print("\nO texto do BDD deve conter pelo menos as colunas: 'ID', 'Cenário', 'Dado que:', 'Quando:', 'Então:'")
            print("\nExemplo:")
            print("| ID | Cenário | Dado que: | Quando: | Então: |")
            print("001,Pesquisa simples, \"O usuário esteja em um navegador de confiança e entrar no site de busca \"https://www.google.com.br/\".\",")
            print("O usuário inserir qualquer termo ou referência que deseja na barra de pesquisa do Google e pressionar enter no teclado.,")
            print("O Google retorna uma página com diversos links referente a sua pesquisa.")

# Inicia a execução do programa chamando a função principal se o script for executado diretamente
if __name__ == "__main__":
    main()